In [1]:
import torch
import os
from ultralytics import YOLO
import cv2
import numpy as np

In [2]:
model = YOLO("../models/xpm_annotate.pt")
# model = YOLO("../models/xpm.pt")
# model = YOLO("../models/roboflow_8vs.pt")

## Individual Image

In [3]:
img_path = '../dataset/annotations2/0103924-1.tiff'
output_dir = os.path.join('../compare/test', os.path.basename(img_path).split('.')[0])
og_filename = os.path.basename(img_path) 

In [4]:
results = model.predict(source=img_path, conf=0.5, save=False, save_txt=False)
masks = results[0].masks
boxes = results[0].boxes


image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\dataset\annotations2\0103924-1.tiff: 320x640 1 mandible, 183.2ms
Speed: 6.0ms preprocess, 183.2ms inference, 13.0ms postprocess per image at shape (1, 3, 320, 640)


In [5]:
input_img = cv2.imread(img_path)
input_img = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)

In [6]:
mask_color = (250, 200, 0)

In [28]:
output_dir = "yolo_predictions/yolov8s"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for idx, mask in enumerate(masks):
    m = torch.squeeze(mask.data)

    mask_np = m.cpu().numpy().astype(np.uint8)

    colored_mask = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
    colored_mask[mask_np == 1] = mask_color

    colored_mask = cv2.resize(colored_mask, (input_img.shape[1], input_img.shape[0]))

    alpha = 0.5  
    overlayed_img = cv2.addWeighted(input_img, 1, colored_mask, alpha, 0)

    new_name = og_filename.replace('.jpg', '') + '_overlay' + '.jpg'

    cv2.imwrite(os.path.join(output_dir, new_name), cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))

    cv2.imshow('Image', cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))
    cv2.waitKey(0)
    cv2.destroyAllWindows() 

In [36]:
input_dir = "dataset/box_jpeg"
output_dir = "yolo_predictions/yolov8s/box"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for filename in os.listdir(input_dir):
    if filename.endswith(".jpg"):
        input_img = cv2.imread(os.path.join(input_dir, filename))

        for idx, mask in enumerate(masks):
            m = torch.squeeze(mask.data)

            mask_np = m.cpu().numpy().astype(np.uint8)

            colored_mask = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
            colored_mask[mask_np == 1] = mask_color

            colored_mask = cv2.resize(colored_mask, (input_img.shape[1], input_img.shape[0]))

            alpha = 0.5  
            overlayed_img = cv2.addWeighted(input_img, 1, colored_mask, alpha, 0)

            new_name = filename.replace('.jpg', '') + '_overlay' + '.jpg'

            cv2.imwrite(os.path.join(output_dir, new_name), cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))

## One Directory

In [15]:
input_dir = "../compare/trials/original"
output_dir = "../compare/trials/run3"

In [16]:
image_files = os.listdir(input_dir)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

mask_color = (250, 200, 0)

for image_file in image_files:
    img_path = os.path.join(input_dir, image_file)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    filename = os.path.basename(img_path) 

    results = model.predict(source=img_path, conf=0.5, save=False, save_txt=False)
    masks = results[0].masks

    if masks is not None:  
        boxes = results[0].boxes

        for mask in masks:
            if mask is not None:
                m = torch.squeeze(mask.data)

                mask_np = m.cpu().numpy().astype(np.uint8)

                colored_mask = np.zeros((mask_np.shape[0], mask_np.shape[1], 3), dtype=np.uint8)
                colored_mask[mask_np == 1] = mask_color

                colored_mask = cv2.resize(colored_mask, (img.shape[1], img.shape[0]))

                alpha = 0.5  
                overlayed_img = cv2.addWeighted(img, 1, colored_mask, alpha, 0)

                new_name = filename.replace('.jpg', '') + '_overlay' + '.jpg'

                cv2.imwrite(os.path.join(output_dir, new_name), cv2.cvtColor(overlayed_img, cv2.COLOR_RGB2BGR))


image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\compare\trials\original\1513551-1.tiff: 320x640 1 mandible, 115.5ms
Speed: 2.0ms preprocess, 115.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\compare\trials\original\1514519-1.tiff: 320x640 1 mandible, 103.5ms
Speed: 1.0ms preprocess, 103.5ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\compare\trials\original\1514834-1.tiff: 320x640 1 mandible, 127.1ms
Speed: 2.0ms preprocess, 127.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\compare\trials\original\1516578-1.tiff: 320x640 1 mandible, 121.5ms
Speed: 2.0ms preprocess, 121.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

image 1/1 c:\Users\kimbe\Documents\GitHub\mandible2024\src\..\compare\trials\origin